# ***MODOAP - Détection et extraction d'illustrations dans les pages de journaux / manuels / magazines***


Ce script permet d'utiliser l'algorithme Mask-RCNN pour détecter et extraire les illustrations dans les pages d'un corpus spécifié. 
Il requiert :
  - des poids issus de l'entraînement préalable d'un modèle sur un corpus particulier
  - un corpus de journaux / manuels sur lequel opérer la détection

## 0. Connexion à un compte Google Drive et création de l'architecture

Nécessite de se connecter à son compte Google Drive et d'entrer un code de vérification.

Crée un dossier Outils_Modoap sur le Drive qui servira à stocker les poids utilisés et les objets extraits.

In [ ]:
import os
from google.colab import drive

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Le Drive est déjà monté")

if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Illustrations_extraites"):
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Illustrations_extraites')
if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids"):
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids')
  

## 1. Préparation du corpus et des poids à utiliser

Le corpus doit être sous forme d'un dossier contenant les pages des documents au format image.

Nécessite d'entrer le chemin absolu vers le dossier contenant le corpus sur le drive. 
La racine du Google Drive est */content/drive/My Drive/*

Possibilité de copier/coller le chemin depuis la fenêtre de gauche : *Files -> clic droit sur un dossier -> Copy Path*

Exemple de chemin:

/content/drive/My Drive/Corpus/corpus_pour_detection/ 

Ou bien entrer "exemple" pour télécharger un corpus de démonstration.

Idem pour les poids : il s'agit d'un fichier .h5 contenu dans un dossier.

In [ ]:
dossier_corpus = input("Entrer le chemin absolu du dossier contenant le corpus, ou taper \"exemple\" ")
if dossier_corpus == "exemple" :
  print("téléchargement d'un corpus de démonstration")
  if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_illustration"):
    os.makedirs('/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_illustration')
  %cd /content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_illustration

  !wget https://github.com/cyril521/modoap-seg/raw/master/Datasets/DroitsLibertes_pour_essais_inference/corpus_test_DL1950.zip

  !7z x ./corpus_test_DL1950.zip
  os.remove("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_illustration/corpus_test_DL1950.zip")
  dossier_corpus = "/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_illustration"

poids = input("Entrer le chemin absolu du dossier contenant les poids, ou taper \"exemple\" ")
if poids == "exemple" : 
  print("téléchargement de poids pour la démonstration")
  %cd /content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids
  !wget https://github.com/cyril521/modoap-seg/raw/master/Poids/poidsDL.7z.001
  !wget https://github.com/cyril521/modoap-seg/raw/master/Poids/poidsDL.7z.002
  !wget https://github.com/cyril521/modoap-seg/raw/master/Poids/poidsDL.7z.003
  !7z x ./poidsDL.7z.001
  os.remove("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids/poidsDL.7z.001")
  os.remove("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids/poidsDL.7z.002")
  os.remove("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids/poidsDL.7z.003")
  poids = "/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Poids"

## 2. Configuration de l'algorithme

In [ ]:
import os
%cd
if not os.path.exists("/root/Mask_RCNN"):

  !git clone --quiet https://github.com/matterport/Mask_RCNN.git
  %cd /root/Mask_RCNN
  !pip install -q PyDrive
  !pip install -r requirements.txt
  !python setup.py install
  !cp ~/Mask_RCNN/samples/balloon/balloon.py ./illustration.py
  !sed -i -- 's/balloon/illustration/g' illustration.py
  !sed -i -- 's/Balloon/Illustration/g' illustration.py
 
else : print("L'algorithme est déjà téléchargé")

%tensorflow_version 1.x
!pip install q keras==2.1.5
!pip install q keras==2.1.5

%cd /root/Mask_RCNN/

import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob

ROOT_DIR = os.path.abspath("/")

import warnings
warnings.filterwarnings("ignore")

sys.path.append(ROOT_DIR) 
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import illustration

IMAGE_DIR = dossier_corpus
config = illustration.IllustrationConfig()

class InferenceConfig(config.__class__):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# 3. Lancement du modèle en mode Inférence

In [ ]:
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)
fichierpoids = [file for file in glob.glob(poids+"/*.h5")][-1]
model.load_weights(fichierpoids, by_name=True)

## 4. Détection d'illustrations sur le corpus

Cette étape peut prendre un certain temps en fonction de la taille du corpus.

Génère un fichier .json dans le dossier du corpus contenant les annotations des illustrations

In [ ]:
corpus = [image for image in glob.glob(dossier_corpus+"/*.*")]
class_names = ['BG', 'illustration']
import cv2
import json
dicocorpus = {}

for cheminfichier in corpus :
  nomfichier = str(cheminfichier.split("/")[-1])
  image = cv2.imread(cheminfichier)
  results = model.detect([image], verbose=0)
  r = results[0]
  listeobjets = [vecteur for vecteur in r["rois"]]
  dicoimage={}

  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                              class_names, r['scores'])

  for i in range(len(r["rois"])) :
    vecteur_region = [str(coord) for coord in listeobjets[i]]
    classe = class_names[r["class_ids"][i]]
    score = str(r["scores"][i])
    dicoobjet = {"regions":vecteur_region, "classe" : classe, "score":score}
    dicoimage["objet"+str(i)] = dicoobjet
  dicocorpus[nomfichier] = dicoimage


with open(os.path.join(dossier_corpus, "annotations_illustration.json"), "w") as dic :
  json.dump(dicocorpus, dic)

# 5. Extraction des images en fichiers .jpg

Extrait les illustrations détectées dans le dossier /content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Illustrations_extraites/

In [ ]:
import json

with open(os.path.join(dossier_corpus, "annotations_illustration.json"), "r") as mondic :
  dicocorpus2 = json.load(mondic)

for nomfichier in dicocorpus2 :
  cheminfichier = os.path.join(dossier_corpus, nomfichier)
  print("chemin fichier :", cheminfichier)
  compteur = 0

  image = cv2.imread(cheminfichier)
  for objet in dicocorpus2[nomfichier].values():
    y1 = int(objet["regions"][0])
    y2 = int(objet["regions"][2])
    x1 = int(objet["regions"][1])
    x2 = int(objet["regions"][3])
    imagecropped = image[y1:y2, x1:x2] # [Y1:Y2, X1:X2] Le vecteur est sous forme [Y1 X1 Y2 X2]
    cv2.imwrite("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Illustrations_extraites/"+nomfichier+"_objet"+str(compteur)+"_"+objet["classe"]+".jpg", imagecropped)
    #cv2_imshow(image)
    compteur += 1

